In [1]:
# import opencv
import cv2
# import mediapipe for pose estimation libraries and mediapipe solutions
import mediapipe as mp 
# import numpy for trigonometry
import numpy as np # import numpy
# time library needed for delay
import time
import pandas as pd

#pd.set_option('display.max_columns', 1000)
#pd.set_option('display.max_rows', 1000)

# set up mediapipe
# import drawing_utils for drawing utility
mp_drawing = mp.solutions.drawing_utils 
# import pose for estimation models
mp_pose = mp.solutions.pose

## Pose Estimation Model

<img src="../images/landmarks.png"/>

## Calculate Angles

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # First point (hip)
    b = np.array(b) # Mid point (knee)
    c = np.array(c) # End point (ankle)
    
    # Y.ankle - Y.knee , X.ankle - X.knee  -  Y.knee - Y.hip , X.knee - X. hip
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    #converting radians to degrees
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0: #max angle is 180
        angle = 360-angle
        
    return angle

### Getting limb angles for curved ball frames

In [3]:
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('../video/final_curved.mp4')

# set timer module, time.time() gives time in seconds (float)
start_time = time.time()
print(time.asctime())

# initializing lists for the difference angles
right_ankle = []
left_ankle = []
right_hip = []
left_hip = []
right_knee = []
left_knee = []
right_shoulder = []
left_shoulder = []
right_elbow = []
left_elbow = []

# global variable for ret
# ret is a variable from openCV to check if video is at last frame
# is ret = False, there are no more frames left in the video and program should cut

ret = True

## Setup mediapipe instance
with mp_pose.Pose(smooth_landmarks = True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while True:
        ret, frame = cap.read()
        
        # close program when video ends
        if ret == False:
            cap.release()
            cv2.destroyAllWindows()
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        #region of interest -- range of height, range of length in pixel.
        # Davinci output = 1080 by 1920
        roi = image[50:950, 1300: 1570]
        
        # Extract landmarks
        
        ###########################################################################################
        # RIGHT KNEE ANGLE
        ###########################################################################################
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate angle
            angle_r_knee = "{:.2f}".format(calculate_angle(r_hip, r_knee, r_ankle))
            
            # Visualize angle
            cv2.putText(image, str(angle_r_knee), 
                           tuple(np.multiply(r_knee, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        ###########################################################################################
        # LEFT KNEE ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            angle_l_knee = "{:.2f}".format(calculate_angle(l_hip, l_knee, l_ankle))
            
            # Visualize angle
            cv2.putText(image, str(angle_l_knee), 
                           tuple(np.multiply(l_knee, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass

        ###########################################################################################
        # RIGHT HIP ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate angle
            angle_r_hip = "{:.2f}".format(calculate_angle(r_shoulder, r_hip, r_knee))
            
            # Visualize angle
            cv2.putText(image, str(angle_r_hip), 
                           tuple(np.multiply(r_hip, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass

        ###########################################################################################
        # LEFT HIP ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            # Calculate angle
            angle_l_hip = "{:.2f}".format(calculate_angle(l_shoulder, l_hip, l_knee))
            
            # Visualize angle
            cv2.putText(image, str(angle_l_hip), 
                           tuple(np.multiply(l_hip, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        ###########################################################################################
        # LEFT ANKLE ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            l_foot_i = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            # Calculate angle
            angle_l_ankle = "{:.2f}".format(calculate_angle(l_knee, l_ankle, l_foot_i))
            
            # Visualize angle
            cv2.putText(image, str(angle_l_ankle), 
                           tuple(np.multiply(l_ankle, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        ###########################################################################################
        # RIGHT ANKLE ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            r_foot_i = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            # Calculate angle
            angle_r_ankle = "{:.2f}".format(calculate_angle(r_knee, r_ankle, r_foot_i))
            
            # Visualize angle
            cv2.putText(image, str(angle_r_ankle), 
                           tuple(np.multiply(r_ankle, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        ###########################################################################################
        # RIGHT SHOULDER ANGLE
        ###########################################################################################

        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            r_hip_s = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            
            # Calculate angle
            angle_r_shoulder = "{:.2f}".format(calculate_angle(r_hip, r_shoulder, r_elbow))
            
            # Visualize angle
            cv2.putText(image, str(angle_r_shoulder), 
                           tuple(np.multiply(r_shoulder, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass        
        
        
        
        
        ###########################################################################################
        # LEFT SHOULDER ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_hip_s = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle_l_shoulder = "{:.2f}".format(calculate_angle(l_hip_s, l_shoulder, l_elbow))
            
            # Visualize angle
            cv2.putText(image, str(angle_l_shoulder), 
                           tuple(np.multiply(l_shoulder, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass        
        
        
        
        ###########################################################################################
        
        
        # collect angles only in 5 second intervals
        if time.time() - start_time >= 5: #<---- Check if 5 sec passed
            print('right ankle', angle_r_ankle)
            print('left ankle', angle_l_ankle)
            print('right hip', angle_r_hip)
            print('left hip', angle_l_hip)
            print('right knee', angle_r_knee)
            print('left knee', angle_l_knee)
            print('right shoulder', angle_r_shoulder)
            print('left shoulder', angle_l_shoulder)
            right_ankle.append(angle_r_ankle)
            left_ankle.append(angle_l_ankle)
            right_hip.append(angle_r_hip)
            left_hip.append(angle_l_hip)
            right_knee.append(angle_r_knee)
            left_knee.append(angle_l_knee)
            right_shoulder.append(angle_r_shoulder)
            left_shoulder.append(angle_l_shoulder)
            
            print(time.asctime())
            start_time = time.time() + 1
            

        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        

        cv2.imshow('Mediapipe Feed', image)

        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            print(time.asctime())
            cap.release()
            cv2.destroyAllWindows()
            break


Fri Sep  3 10:10:02 2021
right ankle 133.38
left ankle 128.62
right hip 135.51
left hip 144.76
right knee 170.32
left knee 164.99
right shoulder 17.02
left shoulder 6.16
Fri Sep  3 10:10:07 2021
right ankle 163.24
left ankle 172.08
right hip 145.89
left hip 155.50
right knee 166.51
left knee 169.88
right shoulder 17.54
left shoulder 57.05
Fri Sep  3 10:10:13 2021
right ankle 145.63
left ankle 125.67
right hip 135.86
left hip 146.38
right knee 170.26
left knee 170.45
right shoulder 15.36
left shoulder 47.48
Fri Sep  3 10:10:19 2021
right ankle 158.20
left ankle 154.53
right hip 137.29
left hip 161.98
right knee 176.78
left knee 169.89
right shoulder 45.68
left shoulder 96.55
Fri Sep  3 10:10:25 2021
right ankle 149.33
left ankle 157.45
right hip 135.32
left hip 142.02
right knee 160.26
left knee 157.06
right shoulder 14.47
left shoulder 1.46
Fri Sep  3 10:10:31 2021
right ankle 137.59
left ankle 93.97
right hip 138.73
left hip 140.17
right knee 169.48
left knee 164.60
right shoulder 27.

right ankle 93.10
left ankle 97.18
right hip 129.93
left hip 144.71
right knee 156.74
left knee 164.61
right shoulder 10.02
left shoulder 2.94
Fri Sep  3 10:14:56 2021
right ankle 150.54
left ankle 159.31
right hip 146.77
left hip 161.61
right knee 150.25
left knee 169.24
right shoulder 8.65
left shoulder 33.68
Fri Sep  3 10:15:03 2021
right ankle 137.34
left ankle 144.06
right hip 139.45
left hip 158.56
right knee 176.24
left knee 175.82
right shoulder 0.36
left shoulder 135.49
Fri Sep  3 10:15:09 2021
right ankle 155.73
left ankle 160.40
right hip 147.26
left hip 150.92
right knee 175.85
left knee 170.13
right shoulder 7.54
left shoulder 5.00
Fri Sep  3 10:15:15 2021
right ankle 139.80
left ankle 150.64
right hip 135.92
left hip 153.39
right knee 168.66
left knee 174.86
right shoulder 16.64
left shoulder 128.24
Fri Sep  3 10:15:21 2021
right ankle 132.09
left ankle 152.76
right hip 149.19
left hip 151.35
right knee 167.55
left knee 166.19
right shoulder 4.75
left shoulder 11.81
Fri S

In [4]:
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [5]:
print(right_ankle)

['133.38', '163.24', '145.63', '158.20', '149.33', '137.59', '157.98', '141.85', '136.69', '167.24', '117.46', '117.96', '157.17', '128.43', '176.49', '117.11', '131.83', '139.06', '157.57', '112.21', '100.78', '158.45', '146.11', '130.06', '171.62', '113.03', '165.81', '133.38', '121.85', '90.60', '157.09', '146.90', '142.78', '132.24', '163.12', '158.53', '168.75', '166.31', '135.38', '157.84', '163.81', '141.88', '178.20', '103.67', '147.75', '126.09', '151.13', '128.59', '93.10', '150.54', '137.34', '155.73', '139.80', '132.09', '150.64', '158.88', '127.19', '178.96', '147.08', '150.15', '172.58', '84.11', '99.52', '126.64', '164.75', '140.33', '118.71', '156.78', '135.57', '145.56', '120.38', '130.46', '131.01', '126.83', '103.67', '161.06', '107.70', '136.34', '139.95', '156.89', '81.62', '139.02', '140.41', '140.41', '140.41']


In [6]:
df_curl = pd.DataFrame({"rightankle" : right_ankle, "leftankle" :left_ankle, 
                   "rightknee" : right_knee, "leftknee" : left_knee, 
                   "righthip" : right_hip, "lefthip" : left_hip,
                    "rightshoulder" : right_shoulder, "leftshoulder" : left_shoulder})

In [7]:
df_curl["curve"] = 1

In [8]:
df_curled = df_curl.copy()

In [9]:
df_curled

,rightankle,leftankle,rightknee,leftknee,righthip,lefthip,rightshoulder,leftshoulder,curve
0,133.38,128.62,170.32,164.99,135.51,144.76,17.02,6.16,1
1,163.24,172.08,166.51,169.88,145.89,155.50,17.54,57.05,1
2,145.63,125.67,170.26,170.45,135.86,146.38,15.36,47.48,1
3,158.20,154.53,176.78,169.89,137.29,161.98,45.68,96.55,1
4,149.33,157.45,160.26,157.06,135.32,142.02,14.47,1.46,1
...,...,...,...,...,...,...,...,...,...
80,81.62,172.55,16.06,15.48,162.87,53.29,167.32,176.10,1
81,139.02,164.28,169.56,167.90,171.37,170.91,31.55,57.36,1
82,140.41,155.71,169.33,168.72,169.86,170.96,21.62,35.08,1
83,140.41,155.71,169.33,168.72,169.86,170.96,21.62,35.08,1


In [10]:
df_curled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   rightankle     85 non-null     object
 1   leftankle      85 non-null     object
 2   rightknee      85 non-null     object
 3   leftknee       85 non-null     object
 4   righthip       85 non-null     object
 5   lefthip        85 non-null     object
 6   rightshoulder  85 non-null     object
 7   leftshoulder   85 non-null     object
 8   curve          85 non-null     int64 
dtypes: int64(1), object(8)
memory usage: 6.1+ KB


In [11]:
col = list(df_curled.columns)
col

['rightankle',
 'leftankle',
 'rightknee',
 'leftknee',
 'righthip',
 'lefthip',
 'rightshoulder',
 'leftshoulder',
 'curve']

In [12]:
for i in col:
    df_curl[i] = pd.to_numeric(df_curl[i], errors='coerce')

In [13]:
df_curl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   rightankle     85 non-null     float64
 1   leftankle      85 non-null     float64
 2   rightknee      85 non-null     float64
 3   leftknee       85 non-null     float64
 4   righthip       85 non-null     float64
 5   lefthip        85 non-null     float64
 6   rightshoulder  85 non-null     float64
 7   leftshoulder   85 non-null     float64
 8   curve          85 non-null     int64  
dtypes: float64(8), int64(1)
memory usage: 6.1 KB


In [14]:
df_curl.describe()

,rightankle,leftankle,rightknee,leftknee,righthip,lefthip,rightshoulder,leftshoulder,curve
count,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.0
mean,139.651529,141.752706,163.349294,164.058941,143.631882,150.915765,16.950235,39.742235,1.0
std,21.913422,23.760886,19.960688,18.638521,12.696380,17.391349,23.896914,37.345033,0.0
min,81.620000,52.710000,16.060000,15.480000,84.130000,53.290000,0.180000,0.380000,1.0
25%,128.430000,128.420000,160.260000,160.260000,138.510000,146.150000,5.810000,10.020000,1.0
50%,140.410000,146.280000,167.550000,167.070000,142.880000,152.100000,12.390000,35.080000,1.0
75%,157.170000,157.890000,173.030000,171.730000,149.420000,160.380000,19.930000,56.390000,1.0
max,178.960000,178.800000,179.620000,179.930000,171.370000,178.060000,167.320000,176.100000,1.0


In [15]:
df_curl.to_csv("../data/curve_angles.csv")

### Getting limb angles for straight ball frames

In [16]:
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('../video/straight_flipped.mp4')

# set timer module, time.time() gives time in seconds (float)
start_time = time.time()
print(time.asctime())

# initializing lists for the difference angles
right_ankle = []
left_ankle = []
right_hip = []
left_hip = []
right_knee = []
left_knee = []
right_shoulder = []
left_shoulder = []
right_elbow = []
left_elbow = []

# global variable for ret
# ret is a variable from openCV to check if video is at last frame
# is ret = False, there are no more frames left in the video and program should cut

ret = True

## Setup mediapipe instance
with mp_pose.Pose(smooth_landmarks = True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while True:
        ret, frame = cap.read()
        
        # close program when video ends
        if ret == False:
            cap.release()
            cv2.destroyAllWindows()
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        #region of interest -- range of height, range of length in pixel.
        # Davinci output = 1080 by 1920
        roi = image[50:950, 1300: 1570]
        
        # Extract landmarks
        
        ###########################################################################################
        # RIGHT KNEE ANGLE
        ###########################################################################################
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            # Calculate angle
            angle_r_knee = "{:.2f}".format(calculate_angle(r_hip, r_knee, r_ankle))
            
            # Visualize angle
            cv2.putText(image, str(angle_r_knee), 
                           tuple(np.multiply(r_knee, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        ###########################################################################################
        # LEFT KNEE ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            angle_l_knee = "{:.2f}".format(calculate_angle(l_hip, l_knee, l_ankle))
            
            # Visualize angle
            cv2.putText(image, str(angle_l_knee), 
                           tuple(np.multiply(l_knee, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass

        ###########################################################################################
        # RIGHT HIP ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            # Calculate angle
            angle_r_hip = "{:.2f}".format(calculate_angle(r_shoulder, r_hip, r_knee))
            
            # Visualize angle
            cv2.putText(image, str(angle_r_hip), 
                           tuple(np.multiply(r_hip, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass

        ###########################################################################################
        # LEFT HIP ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            # Calculate angle
            angle_l_hip = "{:.2f}".format(calculate_angle(l_shoulder, l_hip, l_knee))
            
            # Visualize angle
            cv2.putText(image, str(angle_l_hip), 
                           tuple(np.multiply(l_hip, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        ###########################################################################################
        # LEFT ANKLE ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            l_foot_i = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            
            # Calculate angle
            angle_l_ankle = "{:.2f}".format(calculate_angle(l_knee, l_ankle, l_foot_i))
            
            # Visualize angle
            cv2.putText(image, str(angle_l_ankle), 
                           tuple(np.multiply(l_ankle, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        ###########################################################################################
        # RIGHT ANKLE ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            r_foot_i = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            # Calculate angle
            angle_r_ankle = "{:.2f}".format(calculate_angle(r_knee, r_ankle, r_foot_i))
            
            # Visualize angle
            cv2.putText(image, str(angle_r_ankle), 
                           tuple(np.multiply(r_ankle, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        ###########################################################################################
        # RIGHT SHOULDER ANGLE
        ###########################################################################################

        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            r_hip_s = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            
            # Calculate angle
            angle_r_shoulder = "{:.2f}".format(calculate_angle(r_hip, r_shoulder, r_elbow))
            
            # Visualize angle
            cv2.putText(image, str(angle_r_shoulder), 
                           tuple(np.multiply(r_shoulder, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass        
        
        
        
        
        ###########################################################################################
        # LEFT SHOULDER ANGLE
        ###########################################################################################
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_hip_s = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            angle_l_shoulder = "{:.2f}".format(calculate_angle(l_hip_s, l_shoulder, l_elbow))
            
            # Visualize angle
            cv2.putText(image, str(angle_l_shoulder), 
                           tuple(np.multiply(l_shoulder, [1920, 1080]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA
                                )
                       
        except:
            pass        
        
        
        
        ###########################################################################################
        
        
        # collect angles only in 5 second intervals
        if time.time() - start_time >= 5: #<---- Check if 5 sec passed
            print('right ankle', angle_r_ankle)
            print('left ankle', angle_l_ankle)
            print('right hip', angle_r_hip)
            print('left hip', angle_l_hip)
            print('right knee', angle_r_knee)
            print('left knee', angle_l_knee)
            print('right shoulder', angle_r_shoulder)
            print('left shoulder', angle_l_shoulder)
            right_ankle.append(angle_r_ankle)
            left_ankle.append(angle_l_ankle)
            right_hip.append(angle_r_hip)
            left_hip.append(angle_l_hip)
            right_knee.append(angle_r_knee)
            left_knee.append(angle_l_knee)
            right_shoulder.append(angle_r_shoulder)
            left_shoulder.append(angle_l_shoulder)
            
            print(time.asctime())
            start_time = time.time() + 1
            

        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        

        cv2.imshow('Mediapipe Feed', image)

        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            print(time.asctime())
            cap.release()
            cv2.destroyAllWindows()
            break


Fri Sep  3 10:18:36 2021
right ankle 174.74
left ankle 177.54
right hip 176.70
left hip 174.20
right knee 179.82
left knee 179.36
right shoulder 16.51
left shoulder 19.09
Fri Sep  3 10:18:41 2021
right ankle 176.24
left ankle 166.60
right hip 172.87
left hip 179.30
right knee 176.95
left knee 176.17
right shoulder 25.29
left shoulder 14.28
Fri Sep  3 10:18:47 2021
right ankle 169.42
left ankle 169.14
right hip 179.89
left hip 172.45
right knee 174.35
left knee 178.09
right shoulder 14.30
left shoulder 19.86
Fri Sep  3 10:18:53 2021
right ankle 176.42
left ankle 153.92
right hip 162.99
left hip 167.97
right knee 177.27
left knee 167.83
right shoulder 17.74
left shoulder 6.98
Fri Sep  3 10:18:59 2021
right ankle 174.05
left ankle 170.13
right hip 176.88
left hip 175.12
right knee 173.36
left knee 177.71
right shoulder 13.58
left shoulder 12.62
Fri Sep  3 10:19:05 2021
right ankle 173.03
left ankle 176.23
right hip 176.64
left hip 175.55
right knee 173.30
left knee 176.72
right shoulder 1

right ankle 112.16
left ankle 126.82
right hip 145.96
left hip 155.79
right knee 170.00
left knee 169.11
right shoulder 21.45
left shoulder 82.56
Fri Sep  3 10:23:30 2021
right ankle 117.63
left ankle 123.84
right hip 147.27
left hip 155.78
right knee 171.93
left knee 168.27
right shoulder 21.63
left shoulder 82.39
Fri Sep  3 10:23:36 2021
right ankle 117.55
left ankle 138.28
right hip 118.43
left hip 136.50
right knee 128.84
left knee 140.13
right shoulder 2.97
left shoulder 58.31
Fri Sep  3 10:23:42 2021
right ankle 95.62
left ankle 115.51
right hip 140.38
left hip 158.64
right knee 152.95
left knee 165.52
right shoulder 12.57
left shoulder 23.36
Fri Sep  3 10:23:48 2021
right ankle 134.36
left ankle 102.51
right hip 128.90
left hip 158.38
right knee 137.56
left knee 170.21
right shoulder 0.02
left shoulder 18.95
Fri Sep  3 10:23:54 2021
right ankle 109.93
left ankle 109.70
right hip 135.10
left hip 144.80
right knee 146.95
left knee 134.71
right shoulder 1.74
left shoulder 15.70
Fri

In [17]:
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [18]:
df_straight = pd.DataFrame({"rightankle" : right_ankle, "leftankle" :left_ankle, 
                   "rightknee" : right_knee, "leftknee" : left_knee, 
                   "righthip" : right_hip, "lefthip" : left_hip,
                    "rightshoulder" : right_shoulder, "leftshoulder" : left_shoulder})

In [19]:
df_straight

,rightankle,leftankle,rightknee,leftknee,righthip,lefthip,rightshoulder,leftshoulder
0,174.74,177.54,179.82,179.36,176.70,174.20,16.51,19.09
1,176.24,166.60,176.95,176.17,172.87,179.30,25.29,14.28
2,169.42,169.14,174.35,178.09,179.89,172.45,14.30,19.86
3,176.42,153.92,177.27,167.83,162.99,167.97,17.74,6.98
4,174.05,170.13,173.36,177.71,176.88,175.12,13.58,12.62
...,...,...,...,...,...,...,...,...
56,120.93,118.93,143.26,176.05,134.71,168.42,12.87,31.93
57,114.07,128.43,148.91,146.37,136.94,147.08,0.52,12.04
58,87.75,111.62,134.68,163.41,135.18,165.20,3.93,21.76
59,132.14,134.75,166.68,155.63,149.82,144.96,7.52,21.59


In [20]:
df_straight["curve"] = 0

In [21]:
df_str = df_straight.copy()

In [22]:
df_str.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   rightankle     61 non-null     object
 1   leftankle      61 non-null     object
 2   rightknee      61 non-null     object
 3   leftknee       61 non-null     object
 4   righthip       61 non-null     object
 5   lefthip        61 non-null     object
 6   rightshoulder  61 non-null     object
 7   leftshoulder   61 non-null     object
 8   curve          61 non-null     int64 
dtypes: int64(1), object(8)
memory usage: 4.4+ KB


In [23]:
col_s = list(df_str.columns)
col_s

['rightankle',
 'leftankle',
 'rightknee',
 'leftknee',
 'righthip',
 'lefthip',
 'rightshoulder',
 'leftshoulder',
 'curve']

In [24]:
for i in col_s:
    df_str[i] = pd.to_numeric(df_str[i], errors='coerce')

In [25]:
df_str

,rightankle,leftankle,rightknee,leftknee,righthip,lefthip,rightshoulder,leftshoulder,curve
0,174.74,177.54,179.82,179.36,176.70,174.20,16.51,19.09,0
1,176.24,166.60,176.95,176.17,172.87,179.30,25.29,14.28,0
2,169.42,169.14,174.35,178.09,179.89,172.45,14.30,19.86,0
3,176.42,153.92,177.27,167.83,162.99,167.97,17.74,6.98,0
4,174.05,170.13,173.36,177.71,176.88,175.12,13.58,12.62,0
...,...,...,...,...,...,...,...,...,...
56,120.93,118.93,143.26,176.05,134.71,168.42,12.87,31.93,0
57,114.07,128.43,148.91,146.37,136.94,147.08,0.52,12.04,0
58,87.75,111.62,134.68,163.41,135.18,165.20,3.93,21.76,0
59,132.14,134.75,166.68,155.63,149.82,144.96,7.52,21.59,0


In [26]:
df_str.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   rightankle     61 non-null     float64
 1   leftankle      61 non-null     float64
 2   rightknee      61 non-null     float64
 3   leftknee       61 non-null     float64
 4   righthip       61 non-null     float64
 5   lefthip        61 non-null     float64
 6   rightshoulder  61 non-null     float64
 7   leftshoulder   61 non-null     float64
 8   curve          61 non-null     int64  
dtypes: float64(8), int64(1)
memory usage: 4.4 KB


In [27]:
df_str.describe()

,rightankle,leftankle,rightknee,leftknee,righthip,lefthip,rightshoulder,leftshoulder,curve
count,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.0
mean,129.206066,137.818197,162.162131,166.332295,145.348525,157.414262,9.727377,46.000164,0.0
std,27.410322,24.214714,12.954922,11.057754,12.735029,11.447278,8.511769,29.671175,0.0
min,80.790000,83.500000,127.170000,134.710000,115.900000,130.750000,0.020000,2.960000,0.0
25%,108.070000,118.930000,156.310000,160.680000,137.800000,150.310000,3.350000,22.190000,0.0
50%,120.340000,136.330000,165.500000,167.830000,144.730000,157.440000,6.380000,35.110000,0.0
75%,157.690000,153.920000,171.460000,176.560000,148.980000,165.200000,15.270000,74.700000,0.0
max,176.420000,178.300000,179.820000,179.440000,179.890000,179.300000,43.100000,113.670000,0.0


In [28]:
df_str.to_csv("../data/straight_angles.csv")